In [192]:
!pip3 install statsmodels
!pip3 install vaderSentiment
!pip3 install nbconvert
!pip3 install tabulate

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [193]:
import numpy as np
import pandas as pd

from IPython.display import display

from datetime import date, timedelta
import datetime

import matplotlib.dates as mdates
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

from sklearn.preprocessing import LabelEncoder

import statsmodels.api as sm
import statsmodels.formula.api as smf
import vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import csv

from tabulate import tabulate


In [194]:
current_date = datetime.datetime.strptime("2020-08-22", "%Y-%m-%d").date()

def convertToTime(row, columnName):
    return datetime.datetime.strptime(row[columnName], "%Y-%m-%d").date()

def convertToDate(row, columnName):
    return datetime.datetime.strptime(row[columnName], "%Y-%m-%d %H:%M:%S").date()
    
def update_end_date(row, columnName, latestDate):
    if row[columnName] == row[columnName]:
        return row[columnName]
    else:
        return latestDate

def getDays(row, beginColumnName, endColumnName):
    v = datetime.datetime.strptime(str(row[endColumnName]), "%Y-%m-%d").date() - datetime.datetime.strptime(row[beginColumnName], "%Y-%m-%d").date()
    return v.days

def calculate_recency(row, columnName):
    return (current_date - datetime.datetime.strptime(str(row[columnName]), "%Y-%m-%d").date()).days

def lookup_index(row, columnName, array):
    if(row[columnName] not in array):
        return -1
    return array.index(row[columnName]) + 1

def colour_life_events(row):
    colours = {'personal':'lightcoral', 'health':'orange', 'work':'lightgreen', 'financial':'teal', 'weather':'blueviolet', 'societal':'navy','other':'skyblue'}
    return colours[row['life_event_type']]

def remove_rows(base_df, other_df):
    modified_df = other_df.drop(other_df[other_df['snapshot_id'] not in base_df['snapshot_id'].values].index)
    return modified_df

def fix_signficance(row):
    if('significance' in row['valence']):
        return row['valence']
    else:
        return row['significance']

def fix_valence(row):
    if('significance' in row['significance']):
        return row['valence']
    else:
        return row['significance']  

def get_broad_category(row, categories, column_name):
    if(row[column_name] in categories):
        return categories[row[column_name]]
    return "UNKNOWN"
    
def compute_sentiment(row):
    post = row['Text']
    analyzer = SentimentIntensityAnalyzer()
    vs = analyzer.polarity_scores(post)
    sentiment = 0
    if (vs["neu"]>0.8):
        sentiment = 0
    elif (vs["pos"]==vs["neg"]):
        sentiment = 0
    elif (vs["pos"]>vs["neg"]):
        sentiment = 1
    elif (vs["neg"]>vs["neu"]):
        sentiment = -1
    return sentiment

def convert_valence_to_sentiment(row):
    valence = row['valence']
    retVal = 0
    if (valence == 'Neither Positive or Negative'):
        retVal = 0
    elif("Positive" in valence):
        retVal = 1
    elif("Negative" in valence):
        retVal = -1
    return retVal

def update_education_level(row, columnName):
    if 'college' in row[columnName].lower():
        return "College Degree"
    elif 'doctoral' in row[columnName].lower():
        return "Doctoral Degree"
    elif 'master' in row[columnName].lower() or 'grad' in row[columnName].lower():
        return "Graduate Degree"
    elif 'hs' in row[columnName].lower() or 'high school' in row[columnName].lower():
        return "High School"
    
    return row[columnName]

In [195]:
control_variables             = 'shipley_vocab + shipley_abs + openness + conscientiousness + extraversion + agreeableness + neuroticism + pos_affect + neg_affect + stai_trait + education_level + psqi + age + gender'
combined_control_variables    = 'shipley_vocab + shipley_abs + openness + conscientiousness + extraversion + agreeableness + neuroticism + pos_affect + neg_affect + stai_trait + education_level + psqi + age + gender + data_type'

sr_life_event_variables       = 'Anticipation + LifeEventFamily + valence + recency + status + Intimacy + Scope + significance_label'
sm_life_event_variables       = 'Anticipation + LifeEventFamily + valence + recency + status + Intimacy + Scope'
combined_life_event_variables = 'Anticipation + LifeEventFamily + valence + recency + status + Intimacy + Scope + data_type'

sr_all_variables     = 'shipley_vocab + shipley_abs + openness + conscientiousness + extraversion + agreeableness + neuroticism + pos_affect + neg_affect + stai_trait + education_level + psqi + age + gender + Anticipation + LifeEventFamily + valence + recency + status + Intimacy + Scope + significance_label'
sm_all_variables     = 'shipley_vocab + shipley_abs + openness + conscientiousness + extraversion + agreeableness + neuroticism + pos_affect + neg_affect + stai_trait + education_level + psqi + age + gender + Anticipation + LifeEventFamily + valence + recency + status + Intimacy + Scope'
combined_all_variables        = 'shipley_vocab + shipley_abs + openness + conscientiousness + extraversion + agreeableness + neuroticism + pos_affect + neg_affect + stai_trait + education_level + psqi + age + gender + Anticipation + LifeEventFamily + valence + recency + status + Intimacy + Scope + data_type'
# combined_sm_sr_all_variables  = 'shipley_vocab_sm + shipley_vocab_sr + shipley_abs_sm + shipley_abs_sr + openness_sm + openness_sr + conscientiousness_sm + conscientiousness_sr + extraversion_sm + extraversion_sr + agreeableness_sm + agreeableness_sr + neuroticism_sm + neuroticism_sr + pos_affect_sm + pos_affect_sr + neg_affect_sm + neg_affect_sr + stai_trait_sm + stai_trait_sr + education_level_sm + education_level_sr + psqi_sm + psqi_sr + age_sm + age_sr + gender_sm + gender_sr + Anticipation_sm + Anticipation_sr + LifeEventFamily_sm + LifeEventFamily_sr + valence_sm + valence_sr + Intimacy_sm + Intimacy_sr + Scope_sm + Scope_sr + data_type'

In [196]:
def load_demographics_data():
    demographics_data = pd.read_csv('data/igtbs_demographics_complete.csv', parse_dates=True)
    demographics_data = demographics_data[['age','gender','snapshot_id', 'shipley.vocab', 'shipley.abs', 'openness', 'conscientiousness', 'extraversion', 'agreeableness', 'neuroticism','pos.affect','neg.affect','stai.trait','psqi','educ']]
    demographics_data['education_level'] = demographics_data.apply(update_education_level, columnName='educ', axis=1)
    demographics_data = demographics_data.drop(columns=['educ'])
    return demographics_data

In [197]:
load_demographics_data()

,age,gender,snapshot_id,shipley.vocab,shipley.abs,openness,conscientiousness,extraversion,agreeableness,neuroticism,pos.affect,neg.affect,stai.trait,psqi,education_level
0,39.0,Female,11156103277197680506,32.0,15.0,4.750000,4.666667,3.666667,4.416667,2.333333,38.0,15.0,39.0,7.0,College Degree
1,39.0,Female,13346780717560972831,38.0,19.0,3.500000,4.833333,3.000000,4.916667,1.833333,41.0,16.0,28.0,8.0,Graduate Degree
2,25.0,Female,12932409338132389043,32.0,19.0,4.916667,4.083333,3.750000,4.000000,4.083333,20.0,24.0,55.0,10.0,Graduate Degree
3,33.0,Male,10385244076367909694,34.0,17.0,3.416667,2.166667,2.916667,3.000000,2.000000,25.0,24.0,51.0,9.0,Graduate Degree
4,37.0,Female,10671464268435700062,39.0,15.0,4.500000,5.000000,3.916667,3.750000,3.000000,36.0,16.0,35.0,3.0,College Degree
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749,44.0,Male,10033784408945437200,34.0,17.0,3.500000,4.250000,2.750000,4.500000,2.166667,30.0,13.0,35.0,4.0,Graduate Degree
750,51.0,Male,13257691313989387573,34.0,20.0,4.500000,2.666667,2.750000,4.250000,2.833333,35.0,17.0,41.0,5.0,Graduate Degree
751,44.0,Female,13370808519384713343,35.0,18.0,3.500000,4.250000,4.333333,3.250000,1.083333,39.0,12.0,25.0,8.0,Graduate Degree
752,31.0,Female,12173290729829297463,32.0,17.0,3.583333,4.666667,2.916667,3.416667,3.166667,38.0,18.0,31.0,3.0,College Degree


In [198]:
def load_survey_categories():
    df_self_reported_categories = pd.read_csv('data/Life Events Categories Mapping - Self-Reported Categories.csv')
    return df_self_reported_categories

def load_survey_data_without_categories():
    df_survey = pd.read_csv('data/Superimposed/LifeEvents_Curated_non_blinded.csv', parse_dates=True)    
    df_survey = df_survey[['snapshot_id', 'description','UpdatedBeginDate', 'UpdatedEndDate','life_event_type', 'work_perf_impact', 'significance','valence', 'ended_or_ongoing']]

    # Date manipulation
    latest_date = max(datetime.datetime.strptime(str(x), "%Y-%m-%d").date() if x == x else datetime.date.min for x in df_survey['UpdatedEndDate'])
    latest_date = max(latest_date, max(datetime.datetime.strptime(str(x), "%Y-%m-%d").date() if x == x else datetime.date.min for x in df_survey['UpdatedBeginDate']))
    df_survey = df_survey.drop(df_survey[df_survey['UpdatedBeginDate'].isnull() == True].index)
    df_survey['UpdatedEndDate'] = df_survey.apply(update_end_date, columnName='UpdatedEndDate', latestDate=latest_date, axis=1)
    df_survey['num_of_days'] = df_survey.apply(getDays, endColumnName='UpdatedEndDate', beginColumnName='UpdatedBeginDate', axis=1)
    df_survey['UpdatedBeginDate_time'] = df_survey.apply(convertToTime, columnName='UpdatedBeginDate', axis=1)
    df_survey['recency'] = df_survey.apply(calculate_recency, columnName='UpdatedEndDate', axis=1)
    
    # Update values for valence and significance
    df_survey.replace({'valence': {np.nan: 'Neither Positive or Negative'}, 'significance': {np.nan: 'Neither Positive or Negative'}}, inplace=True)
    df_survey['fixed_signficance'] = df_survey.apply(fix_signficance, axis = 1)
    df_survey['fixed_valence'] = df_survey.apply(fix_valence, axis = 1)
    df_survey = df_survey.drop(columns = ['valence', 'significance'])
    df_survey = df_survey.rename(columns={"fixed_signficance": "significance", "fixed_valence": "valence"})
    df_survey['valence'] = df_survey.apply(convert_valence_to_sentiment, axis=1)
    
    # Select columns we are interested in
    df_survey = df_survey[['snapshot_id', 'description', 'UpdatedBeginDate', 'UpdatedEndDate', 'significance', 'valence', 'ended_or_ongoing', 'recency']]

    # Label encoding for significance
    le_significance = LabelEncoder()
    le_significance.fit(df_survey['significance'].values)
    df_survey['significance_label'] = df_survey.apply(lambda x: le_significance.transform([x['significance']])[0], axis=1)
    df_survey = df_survey.drop(columns=['significance'])

    return df_survey

def load_survey_data():
    df_survey_without_categories = load_survey_data_without_categories()
    df_self_reported_categories = load_survey_categories()
    df_survey = pd.merge(df_survey_without_categories, df_self_reported_categories, how="inner", left_on="description", right_on="SR_LifeEvent")
    df_survey = df_survey.drop(columns=['description', 'SR_LifeEvent', 'LifeEventFinal', 'LifeEventFamily2'])
    df_survey = df_survey.rename(columns={'ended_or_ongoing':'status'})

    return df_survey

In [199]:
load_survey_data()

,snapshot_id,UpdatedBeginDate,UpdatedEndDate,valence,status,recency,significance_label,LifeEventFamily,Anticipation,Intimacy,Scope
0,12516215411311181588,2018-07-07,2018-07-28,1,Ended,756,4,Personal,Anticipated,2,2
1,12672283172047217763,2018-09-28,2018-10-15,1,Ended,677,4,Personal,Anticipated,2,2
2,11485911854465942509,2018-10-24,2019-01-02,1,Ended,598,2,Personal,Anticipated,2,2
3,11829059645538974156,2018-12-20,2018-12-31,1,Ended,600,0,Personal,Anticipated,2,2
4,12516215411311181588,2018-08-30,2018-09-04,1,Ended,718,2,Personal,Anticipated,2,2
...,...,...,...,...,...,...,...,...,...,...,...
1122,13146988918954476898,2018-05-01,2018-06-01,1,Ended,813,4,Financial,Anticipated,3,3
1123,12204601337084291188,2018-07-15,2019-04-10,1,NaN,500,5,Work,Unanticipated,3,3
1124,12718614204893805965,2019-02-01,2019-03-15,-1,Ended,526,5,Local,Unanticipated,1,1
1125,9883624450358368517,2019-01-29,2019-02-03,-1,Ended,566,5,Local,Unanticipated,1,1


In [200]:
def load_social_media_categories():
    df_social_media_categories = pd.read_csv('data/Life Events Categories Mapping - Social Media Categories-2.csv')
    return df_social_media_categories

def load_social_media_data_without_categories():
    df_social_media_data = pd.read_csv('data/Superimposed/Facebook Data For Life Events-Combined - FB Data.csv')
    df_social_media_data = df_social_media_data[['snapshot_id', 'created_time', 'Text', 'final_life_event_category_2', 'ended/ongoing']]
    df_social_media_data = df_social_media_data.replace({'PostiveMove':'PositiveMove', 'Negative Move':'NegativeMove'})
    df_social_media_data = df_social_media_data.drop(df_social_media_data[((df_social_media_data['final_life_event_category_2'].isnull() == True))].index)
    df_social_media_data['created_date'] = df_social_media_data.apply(convertToDate, columnName='created_time', axis=1)
    df_social_media_data['valence'] = df_social_media_data.apply(compute_sentiment, axis=1)
    df_social_media_data = df_social_media_data.drop(columns=['created_time','Text'])
    df_social_media_data['recency'] = df_social_media_data.apply(calculate_recency, columnName='created_date', axis=1)
    return df_social_media_data

def load_social_media_data():
    df_social_media_data = load_social_media_data_without_categories()
    df_social_media_categories = load_social_media_categories()
    df_social_media_data_with_categories = pd.merge(df_social_media_data, df_social_media_categories, how="inner", left_on='final_life_event_category_2', right_on='SM_LifeEvent')
    df_social_media_data_with_categories = df_social_media_data_with_categories.drop(columns=['final_life_event_category_2','SM_LifeEvent','LifeEventFamily2','Comments','SignificanceRank'])
    df_social_media_data_with_categories = df_social_media_data_with_categories.rename(columns={'ended/ongoing':'status'})
    return df_social_media_data_with_categories

In [201]:
load_social_media_data()

,snapshot_id,status,created_date,valence,recency,LifeEventFamily,Anticipation,Intimacy,Scope
0,12840161237957324034,ongoing,2018-03-16,0,890,Personal,Anticipated,2,2
1,11156103277197680506,ongoing,2018-07-31,1,753,Personal,Anticipated,2,2
2,11329586216091316598,ongoing,2018-09-08,0,714,Personal,Anticipated,2,2
3,13543308681200334026,ongoing,2018-07-22,0,762,Personal,Anticipated,2,2
4,12676581714475000627,ongoing,2018-04-25,0,850,Personal,Anticipated,2,2
...,...,...,...,...,...,...,...,...,...
1975,10729524556661174446,ongoing,2018-12-19,0,612,School,Unanticipated,3,2
1976,9767765376628075157,ongoing,2018-07-27,0,757,Health,Anticipated,3,3
1977,9767765376628075157,ongoing,2018-10-23,0,669,Health,Anticipated,3,3
1978,9767765376628075157,ended,2018-09-19,1,703,Health,Unanticipated,3,3


In [202]:
def load_dailies_data():
    df_dailies = pd.read_csv('data/Superimposed/dailies_scores.csv', low_memory=False)
    df_dailies = df_dailies[['snapshot_id','day', 'alc_status', 'alc.quantity.d', 'anxiety.d', 'pos.affect.d', 'neg.affect.d','sleep.d', 'stress.d']]
    df_dailies['day_time'] = df_dailies.apply(convertToTime, columnName='day', axis=1)
    df_dailies = df_dailies.rename(columns={'alc.quantity.d': 'alc_quantity',
    'anxiety.d': 'anxiety',
    'pos.affect.d': 'pos_affect',
    'neg.affect.d': 'neg_affect',
    'sleep.d': 'sleep',
    'stress.d': 'stress'})
    return df_dailies

In [203]:
load_dailies_data()

,snapshot_id,day,alc_status,alc_quantity,anxiety,pos_affect,neg_affect,sleep,stress,day_time
0,10003528337325499062,2018-03-01,yes,3.0,2.0,11.0,6.0,6.0,2.0,2018-03-01
1,10003528337325499062,2018-03-02,no,0.0,2.0,13.0,5.0,8.0,2.0,2018-03-02
2,10003528337325499062,2018-03-04,no,0.0,1.0,5.0,5.0,9.0,1.0,2018-03-04
3,10003528337325499062,2018-03-06,no,0.0,1.0,15.0,5.0,8.0,1.0,2018-03-06
4,10003528337325499062,2018-03-09,no,0.0,2.0,11.0,5.0,8.0,1.0,2018-03-09
...,...,...,...,...,...,...,...,...,...,...
38592,9999689421615292586,2018-06-23,NaN,NaN,1.0,8.0,5.0,NaN,1.0,2018-06-23
38593,9999689421615292586,2018-06-25,NaN,NaN,1.0,5.0,5.0,NaN,1.0,2018-06-25
38594,9999689421615292586,2018-06-28,NaN,NaN,2.0,12.0,5.0,NaN,2.0,2018-06-28
38595,9999689421615292586,2018-06-30,NaN,NaN,1.0,8.0,5.0,NaN,1.0,2018-06-30


In [204]:
def calculate_mean_dailies_data(df_dailies, dependent_variable):
    df_dailies = df_dailies.drop(df_dailies[((df_dailies[dependent_variable].isnull()))].index)
    mean_by_snapshot_id_df = df_dailies.groupby('snapshot_id', as_index=False).mean()

    return mean_by_snapshot_id_df

In [205]:
def build_df_for_regression_survey(dependent_variable):
    try:
        return pd.read_csv('Linear Regression/linear_regression_survey_'+dependent_variable+'.csv')
    except:
        
        df_survey = load_survey_data()
        df_demographics = load_demographics_data()
        df_dailies = load_dailies_data()
        mean_by_snapshot_id_df = calculate_mean_dailies_data(df_dailies, dependent_variable)
        
        mean_value_df = mean_by_snapshot_id_df[['snapshot_id', dependent_variable]]
        merged_data = pd.merge(df_survey, df_demographics, how="inner", on=["snapshot_id"])

        final_input_dataset = []

        # Use average values for a given snapshot id if the data for dependent variable is not available.
        # Since the life events span over a period of time, add one row for each day of life event.
        for i in merged_data.values:

            end_date = datetime.datetime.strptime(str(i[2]), "%Y-%m-%d").date()
            start_date = datetime.datetime.strptime(i[1], "%Y-%m-%d").date()
            days = (end_date - start_date).days
            avg_of_dependent_variable_list = list(mean_value_df[(mean_value_df['snapshot_id'] == i[0])][dependent_variable].values)
            for j in range(days):
                day = start_date+timedelta(days=j)
                x = list(df_dailies[((df_dailies['snapshot_id'] == i[0]) & (df_dailies['day_time'] == day))][dependent_variable])

                avg_of_dependent_variable = 0
                actual_value = 0
                if(len(x) != 0):
                    avg_of_dependent_variable = x[0]
                    actual_value = avg_of_dependent_variable
                else:
                    avg_of_dependent_variable = avg_of_dependent_variable_list[0]
                s_d = []
                s_d.extend(i)
                s_d.append(actual_value) 
                s_d.append(avg_of_dependent_variable)
                s_d.append(day)
                final_input_dataset.append(s_d)

        final_input_dataset = pd.DataFrame(final_input_dataset, columns=[
            'snapshot_id', 'BeginDate', 'EndDate', 'valence', 'status', 'recency','significance_label', 'LifeEventFamily',
            'Anticipation', 'Intimacy', 'Scope', 'age', 'gender', 'shipley_vocab', 'shipley_abs', 'openness', 'conscientiousness',
            'extraversion', 'agreeableness', 'neuroticism', 'pos_affect', 'neg_affect', 'stai_trait', 'psqi', 'education_level',
            dependent_variable, dependent_variable+'_average', 'exact_day'])     

        X_input = final_input_dataset.drop(columns=['snapshot_id', 'BeginDate', 'EndDate', 'exact_day', dependent_variable])
        X_input = X_input.dropna()

        X_input = X_input.rename(columns={dependent_variable+'_average':dependent_variable})
        X_input.to_csv('Linear Regression/linear_regression_survey_'+dependent_variable+'.csv', index=False)
        return X_input

In [206]:
def build_df_for_regression_social_media(dependent_variable):
    try:
        return pd.read_csv('Linear Regression/linear_regression_social_media_'+dependent_variable+'.csv')
    except:
        df_social_media = load_social_media_data()
        df_demographics = load_demographics_data()
        df_dailies = load_dailies_data()
        mean_by_snapshot_id_df = calculate_mean_dailies_data(df_dailies, dependent_variable)
        
        mean_value_df = mean_by_snapshot_id_df[['snapshot_id', dependent_variable]]
        merged_data = pd.merge(df_social_media, df_demographics, how="inner", on=["snapshot_id"])

        final_input_dataset = []

        #Use average values for a given snapshot id if the data for dependent variable is not available.
        for i in merged_data.values:
            day = i[2]
            avg_of_dependent_variable_list = list(mean_value_df[(mean_value_df['snapshot_id'] == i[0])][dependent_variable].values)

            x = list(df_dailies[((df_dailies['snapshot_id'] == i[0]) & (df_dailies['day_time'] == day))][dependent_variable])
            avg_of_dependent_variable = 0
            actual_value = 0
            if(len(x) != 0):
                avg_of_dependent_variable = x[0]
                actual_value = avg_of_dependent_variable
            else:
                avg_of_dependent_variable = avg_of_dependent_variable_list[0]
            s_d = []
            s_d.extend(i)
            s_d.append(actual_value) 
            s_d.append(avg_of_dependent_variable)
            s_d.append(day)
            final_input_dataset.append(s_d)

        final_input_dataset = pd.DataFrame(final_input_dataset, columns=[
            'snapshot_id', 'status', 'created_date', 'valence', 'recency', 'LifeEventFamily',
            'Anticipation', 'Intimacy', 'Scope', 'age', 'gender', 'shipley_vocab', 'shipley_abs', 'openness', 'conscientiousness',
            'extraversion', 'agreeableness', 'neuroticism', 'pos_affect', 'neg_affect', 'stai_trait', 'psqi', 'education_level',
            dependent_variable, dependent_variable+'_average', 'exact_day'])     

        X_input = final_input_dataset.drop(columns=['snapshot_id', 'created_date', 'exact_day', dependent_variable])
        X_input = X_input.dropna()
        X_input = X_input.rename(columns={dependent_variable+'_average':dependent_variable})
        X_input.to_csv('Linear Regression/linear_regression_social_media_'+dependent_variable+'.csv', index=False)

        return X_input

In [207]:
def regression_on_survey(dependent_variable, formula):
    X_input = build_df_for_regression_survey(dependent_variable)

    mod = smf.ols(formula=dependent_variable+'~'+formula, data=X_input)
    res = mod.fit()
    return res

In [208]:
def regression_on_social_media(dependent_variable, formula):
    X_input = build_df_for_regression_social_media(dependent_variable)
    
    mod = smf.ols(formula=dependent_variable+'~'+formula, data=X_input)
    res = mod.fit()
    return res

In [209]:
def regression_on_combined_survey_social_media(dependent_variable, formula):
    df_survey = build_df_for_regression_survey(dependent_variable)
    df_social_media = build_df_for_regression_social_media(dependent_variable)
    
    df_survey['data_type'] = 'Survey'
    df_social_media['data_type'] = 'Social Media'
    
    df_survey = df_survey.drop(columns=['significance_label'])

    X_input = pd.concat([df_survey, df_social_media])

    mod = smf.ols(formula=dependent_variable+'~'+formula, data=X_input)
    res = mod.fit()
    return res

In [210]:
# def regression_on_survey_social_media_separate_variables(dependent_variable, formula):
#     df_survey = build_df_for_regression_survey(dependent_variable)
#     df_social_media = build_df_for_regression_social_media(dependent_variable)
#     df_survey = df_survey.drop(columns=['significance_label'])

#     df_survey = df_survey.rename(columns=lambda x:  x+'_sr' if x != dependent_variable else x)
#     df_social_media = df_social_media.rename(columns=lambda x: x+'_sm' if x != dependent_variable else x)
    
#     df_survey['data_type'] = 'Survey'
#     df_social_media['data_type'] = 'Social Media'
    
#     X_input = df_survey.append(df_social_media)
#     X_input = X_input.fillna(0)
#     mod = smf.ols(formula=dependent_variable+'~'+formula, data=X_input)
#     res = mod.fit()
#     return res

# regression_on_survey_social_media_separate_variables('stress', combined_sm_sr_all_variables).summary()

In [211]:
def baseline_control_variables(dependent_variable):
    return regression_on_combined_survey_social_media(dependent_variable, combined_control_variables)

In [212]:
def baseline_life_event_variables(dependent_variable):
    return regression_on_combined_survey_social_media(dependent_variable, combined_life_event_variables)

In [213]:
def baseline_life_event_variables_sr(dependent_variable):
    return regression_on_survey(dependent_variable, sr_life_event_variables)

In [214]:
def baseline_life_event_variables_sm(dependent_variable):
    return regression_on_social_media(dependent_variable, sm_life_event_variables)

In [215]:
def regression_all_variables(dependent_variable):
    return regression_on_combined_survey_social_media(dependent_variable, combined_all_variables)

In [216]:
def regression_all_variables_sr(dependent_variable):
    return regression_on_survey(dependent_variable, sr_all_variables)

In [217]:
def regression_all_variables_sm(dependent_variable):
    return regression_on_social_media(dependent_variable, sm_all_variables)

# Regression Results

### STRESS

In [218]:
dependent_variable = 'stress'

In [219]:
# Establish baseline using CONTROL VARIABLES.
# Data includes both survey and social media.
baseline_control_variables(dependent_variable).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 stress   R-squared:                       0.195
Model:                            OLS   Adj. R-squared:                  0.195
Method:                 Least Squares   F-statistic:                     422.9
Date:                Tue, 20 Sep 2022   Prob (F-statistic):               0.00
Time:                        22:38:08   Log-Likelihood:                -21428.
No. Observations:               29642   AIC:                         4.289e+04
Df Residuals:                   29624   BIC:                         4.304e+04
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
======================================================================================================
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Intercept                              0.6990      0.061     11.501      0.000       0.580       0.818
education_level[T.Doctoral Degree]     0.3003      0.027     10.939      0.000       0.247       0.354
education_level[T.Graduate Degree]     0.1691      0.006     26.768      0.000       0.157       0.181
education_level[T.High School]        -0.1705      0.054     -3.142      0.002      -0.277      -0.064
gender[T.Male]                         0.0408      0.007      5.822      0.000       0.027       0.055
data_type[T.Survey]                    0.1149      0.012      9.447      0.000       0.091       0.139
shipley_vocab                         -0.0100      0.001    -10.763      0.000      -0.012      -0.008
shipley_abs                            0.0096      0.001      7.903      0.000       0.007       0.012
openness                               0.0308      0.005      5.786      0.000       0.020       0.041
conscientiousness                      0.0608      0.005     11.144      0.000       0.050       0.071
extraversion                           0.0680      0.005     13.186      0.000       0.058       0.078
agreeableness                          0.0927      0.006     16.148      0.000       0.081       0.104
neuroticism                            0.0696      0.006     10.771      0.000       0.057       0.082
pos_affect                            -0.0138      0.001    -18.394      0.000      -0.015      -0.012
neg_affect                             0.0154      0.001     16.180      0.000       0.014       0.017
stai_trait                             0.0094      0.001     14.747      0.000       0.008       0.011
psqi                                   0.0277      0.001     20.740      0.000       0.025       0.030
age                                   -0.0013      0.000     -3.378      0.001      -0.002      -0.001
==============================================================================
Omnibus:                      983.325   Durbin-Watson:                   0.438
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2364.034
Skew:                           0.164   Prob(JB):                         0.00
Kurtosis:                       4.344   Cond. No.                     1.62e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.62e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [220]:
# Establish baseline using LIFE EVENT VARIABLES.
# Only SURVEY data is used.
baseline_life_event_variables_sr(dependent_variable).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 stress   R-squared:                       0.029
Model:                            OLS   Adj. R-squared:                  0.029
Method:                 Least Squares   F-statistic:                     69.79
Date:                Tue, 20 Sep 2022   Prob (F-statistic):          6.58e-169
Time:                        22:38:08   Log-Likelihood:                -22162.
No. Observations:               27667   AIC:                         4.435e+04
Df Residuals:                   27654   BIC:                         4.446e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
=================================================================================================
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept                         1.8750      0.034     54.389      0.000       1.807       1.943
Anticipation[T.Unanticipated]     0.0027      0.009      0.311      0.756      -0.014       0.020
LifeEventFamily[T.Health]        -0.0924      0.017     -5.377      0.000      -0.126      -0.059
LifeEventFamily[T.Local]          0.1497      0.021      7.007      0.000       0.108       0.192
LifeEventFamily[T.Personal]       0.0223      0.014      1.648      0.099      -0.004       0.049
LifeEventFamily[T.School]         0.2970      0.022     13.261      0.000       0.253       0.341
LifeEventFamily[T.Work]           0.0199      0.016      1.232      0.218      -0.012       0.052
status[T.Ongoing]                 0.0483      0.008      5.816      0.000       0.032       0.065
valence                          -0.0490      0.004    -12.367      0.000      -0.057      -0.041
recency                           0.0005   3.94e-05     11.779      0.000       0.000       0.001
Intimacy                          0.0863      0.008     10.302      0.000       0.070       0.103
Scope                            -0.0673      0.009     -7.391      0.000      -0.085      -0.049
significance_label               -0.0034      0.001     -2.305      0.021      -0.006      -0.001
==============================================================================
Omnibus:                       99.958   Durbin-Watson:                   0.313
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              135.366
Skew:                           0.003   Prob(JB):                     4.03e-30
Kurtosis:                       3.343   Cond. No.                     7.58e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.58e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [221]:
# Establish baseline using LIFE EVENT VARIABLES.
# Only SOCIAL MEDIA data is used.
baseline_life_event_variables_sm(dependent_variable).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 stress   R-squared:                       0.017
Model:                            OLS   Adj. R-squared:                  0.012
Method:                 Least Squares   F-statistic:                     3.176
Date:                Tue, 20 Sep 2022   Prob (F-statistic):           0.000277
Time:                        22:38:08   Log-Likelihood:                -1924.4
No. Observations:                1975   AIC:                             3873.
Df Residuals:                    1963   BIC:                             3940.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
=================================================================================================
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept                         1.9208      0.241      7.970      0.000       1.448       2.393
Anticipation[T.Unanticipated]     0.0534      0.041      1.317      0.188      -0.026       0.133
LifeEventFamily[T.Health]         0.4741      0.155      3.058      0.002       0.170       0.778
LifeEventFamily[T.Local]          0.2336      0.166      1.408      0.159      -0.092       0.559
LifeEventFamily[T.Personal]       0.1954      0.142      1.373      0.170      -0.084       0.474
LifeEventFamily[T.School]         0.3104      0.166      1.869      0.062      -0.015       0.636
LifeEventFamily[T.Work]           0.3354      0.156      2.150      0.032       0.029       0.641
status[T.ongoing]                 0.0271      0.042      0.648      0.517      -0.055       0.109
valence                          -0.0193      0.029     -0.660      0.509      -0.077       0.038
recency                        -9.23e-05      0.000     -0.522      0.602      -0.000       0.000
Intimacy                         -0.0088      0.037     -0.241      0.810      -0.081       0.063
Scope                            -0.0238      0.050     -0.474      0.636      -0.123       0.075
==============================================================================
Omnibus:                      147.987   Durbin-Watson:                   0.927
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              239.999
Skew:                           0.566   Prob(JB):                     7.67e-53
Kurtosis:                       4.279   Cond. No.                     1.92e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.92e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [222]:
# Establish baseline using both CONTROL and LIFE EVENT VARIABLES.
# Only SURVEY data is used.
regression_all_variables_sr(dependent_variable).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 stress   R-squared:                       0.214
Model:                            OLS   Adj. R-squared:                  0.213
Method:                 Least Squares   F-statistic:                     268.7
Date:                Tue, 20 Sep 2022   Prob (F-statistic):               0.00
Time:                        22:38:09   Log-Likelihood:                -19244.
No. Observations:               27667   AIC:                         3.855e+04
Df Residuals:                   27638   BIC:                         3.878e+04
Df Model:                          28                                         
Covariance Type:            nonrobust                                         
======================================================================================================
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Intercept                              0.8869      0.069     12.944      0.000       0.753       1.021
education_level[T.Doctoral Degree]     0.3047      0.032      9.497      0.000       0.242       0.368
education_level[T.Graduate Degree]     0.1525      0.007     22.085      0.000       0.139       0.166
education_level[T.High School]        -0.6598      0.243     -2.713      0.007      -1.137      -0.183
gender[T.Male]                         0.0262      0.008      3.306      0.001       0.011       0.042
Anticipation[T.Unanticipated]         -0.0009      0.008     -0.113      0.910      -0.017       0.015
LifeEventFamily[T.Health]             -0.0176      0.016     -1.067      0.286      -0.050       0.015
LifeEventFamily[T.Local]               0.1692      0.020      8.537      0.000       0.130       0.208
LifeEventFamily[T.Personal]            0.0099      0.013      0.792      0.428      -0.015       0.035
LifeEventFamily[T.School]              0.2350      0.022     10.822      0.000       0.192       0.278
LifeEventFamily[T.Work]                0.0980      0.015      6.555      0.000       0.069       0.127
status[T.Ongoing]                      0.0244      0.008      2.951      0.003       0.008       0.041
shipley_vocab                         -0.0114      0.001    -11.838      0.000      -0.013      -0.010
shipley_abs                            0.0012      0.001      0.895      0.371      -0.001       0.004
openness                               0.0480      0.006      8.526      0.000       0.037       0.059
conscientiousness                      0.0715      0.006     12.044      0.000       0.060       0.083
extraversion                           0.0573      0.006     10.396      0.000       0.047       0.068
agreeableness                          0.0897      0.006     14.790      0.000       0.078       0.102
neuroticism                            0.0573      0.007      7.941      0.000       0.043       0.071
pos_affect                            -0.0157      0.001    -19.621      0.000      -0.017      -0.014
neg_affect                             0.0169      0.001     16.627      0.000       0.015       0.019
stai_trait                             0.0086      0.001     12.382      0.000       0.007       0.010
psqi                                   0.0246      0.001     17.028      0.000       0.022       0.027
age                                   -0.0036      0.000     -8.695      0.000      -0.004      -0.003
valence                               -0.0580      0.004    -15.103      0.000      -0.066      -0.050
recency                                0.0004   3.74e-05     10.560      0.000       0.000       0.000
Intimacy                               0.0841      0.008     10.611      0.000       0.069      

In [223]:
# Establish baseline using both CONTROL and LIFE EVENT VARIABLES.
# Only SOCIAL MEDIA data is used.
regression_all_variables_sm(dependent_variable).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 stress   R-squared:                       0.254
Model:                            OLS   Adj. R-squared:                  0.244
Method:                 Least Squares   F-statistic:                     24.55
Date:                Tue, 20 Sep 2022   Prob (F-statistic):          7.17e-104
Time:                        22:38:09   Log-Likelihood:                -1652.5
No. Observations:                1975   AIC:                             3361.
Df Residuals:                    1947   BIC:                             3517.
Df Model:                          27                                         
Covariance Type:            nonrobust                                         
======================================================================================================
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Intercept                             -0.0043      0.317     -0.014      0.989      -0.627       0.618
education_level[T.Doctoral Degree]     0.2813      0.065      4.309      0.000       0.153       0.409
education_level[T.Graduate Degree]     0.2222      0.031      7.077      0.000       0.161       0.284
education_level[T.High School]        -0.0509      0.070     -0.726      0.468      -0.189       0.087
gender[T.Male]                        -0.0510      0.030     -1.676      0.094      -0.111       0.009
Anticipation[T.Unanticipated]          0.0771      0.036      2.145      0.032       0.007       0.148
LifeEventFamily[T.Health]              0.4920      0.137      3.595      0.000       0.224       0.760
LifeEventFamily[T.Local]               0.2895      0.147      1.972      0.049       0.002       0.577
LifeEventFamily[T.Personal]            0.2807      0.126      2.231      0.026       0.034       0.527
LifeEventFamily[T.School]              0.3982      0.147      2.710      0.007       0.110       0.686
LifeEventFamily[T.Work]                0.3616      0.138      2.623      0.009       0.091       0.632
status[T.ongoing]                      0.0125      0.037      0.339      0.735      -0.060       0.085
shipley_vocab                          0.0128      0.005      2.787      0.005       0.004       0.022
shipley_abs                            0.0136      0.005      2.625      0.009       0.003       0.024
openness                               0.0284      0.028      1.033      0.302      -0.026       0.082
conscientiousness                     -0.0252      0.026     -0.959      0.337      -0.077       0.026
extraversion                          -0.0615      0.025     -2.442      0.015      -0.111      -0.012
agreeableness                          0.0865      0.027      3.171      0.002       0.033       0.140
neuroticism                            0.1226      0.028      4.322      0.000       0.067       0.178
pos_affect                             0.0016      0.003      0.472      0.637      -0.005       0.008
neg_affect                             0.0205      0.004      4.809      0.000       0.012       0.029
stai_trait                             0.0056      0.003      1.699      0.089      -0.001       0.012
psqi                                   0.0176      0.006      3.067      0.002       0.006       0.029
age                                   -0.0033      0.002     -2.093      0.036      -0.006      -0.000
valence                               -0.0025      0.026     -0.095      0.924      -0.053       0.049
recency                            -5.277e-05      0.000     -0.332      0.740      -0.000       0.000
Intimacy                              -0.0361      0.032     -1.115      0.265      -0.100      

In [224]:
# Establish baseline using both CONTROL and LIFE EVENT VARIABLES.
# Data includes both survey and social media.
regression_all_variables(dependent_variable).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 stress   R-squared:                       0.214
Model:                            OLS   Adj. R-squared:                  0.213
Method:                 Least Squares   F-statistic:                     277.8
Date:                Tue, 20 Sep 2022   Prob (F-statistic):               0.00
Time:                        22:38:10   Log-Likelihood:                -21081.
No. Observations:               29642   AIC:                         4.222e+04
Df Residuals:                   29612   BIC:                         4.247e+04
Df Model:                          29                                         
Covariance Type:            nonrobust                                         
======================================================================================================
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Intercept                              0.5545      0.050     11.050      0.000       0.456       0.653
education_level[T.Doctoral Degree]     0.2952      0.027     10.796      0.000       0.242       0.349
education_level[T.Graduate Degree]     0.1629      0.007     24.319      0.000       0.150       0.176
education_level[T.High School]        -0.2067      0.054     -3.841      0.000      -0.312      -0.101
gender[T.Male]                         0.0190      0.007      2.540      0.011       0.004       0.034
Anticipation[T.Unanticipated]          0.0057      0.008      0.721      0.471      -0.010       0.021
LifeEventFamily[T.Health]             -0.0028      0.016     -0.171      0.864      -0.035       0.029
LifeEventFamily[T.Local]               0.1556      0.019      8.005      0.000       0.118       0.194
LifeEventFamily[T.Personal]            0.0112      0.013      0.892      0.372      -0.013       0.036
LifeEventFamily[T.School]              0.2108      0.021      9.933      0.000       0.169       0.252
LifeEventFamily[T.Work]                0.0909      0.015      6.152      0.000       0.062       0.120
status[T.Ongoing]                      0.0263      0.008      3.220      0.001       0.010       0.042
status[T.ended]                        0.1287      0.028      4.672      0.000       0.075       0.183
status[T.ongoing]                      0.1552      0.021      7.374      0.000       0.114       0.197
data_type[T.Survey]                    0.2706      0.019     14.532      0.000       0.234       0.307
shipley_vocab                         -0.0108      0.001    -11.496      0.000      -0.013      -0.009
shipley_abs                            0.0040      0.001      3.165      0.002       0.002       0.007
openness                               0.0412      0.005      7.642      0.000       0.031       0.052
conscientiousness                      0.0646      0.006     11.370      0.000       0.053       0.076
extraversion                           0.0539      0.005     10.163      0.000       0.043       0.064
agreeableness                          0.0925      0.006     15.840      0.000       0.081       0.104
neuroticism                            0.0535      0.007      7.958      0.000       0.040       0.067
pos_affect                            -0.0147      0.001    -19.014      0.000      -0.016      -0.013
neg_affect                             0.0179      0.001     18.234      0.000       0.016       0.020
stai_trait                             0.0090      0.001     13.379      0.000       0.008       0.010
psqi                                   0.0252      0.001     18.393      0.000       0.023       0.028
age                                   -0.0033      0.000     -8.652      0.000      -0.004      

### SLEEP

In [225]:
dependent_variable = 'sleep'

In [226]:
# Establish baseline using CONTROL VARIABLES.
# Data includes both survey and social media.
baseline_control_variables(dependent_variable).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  sleep   R-squared:                       0.209
Model:                            OLS   Adj. R-squared:                  0.209
Method:                 Least Squares   F-statistic:                     439.2
Date:                Tue, 20 Sep 2022   Prob (F-statistic):               0.00
Time:                        22:38:10   Log-Likelihood:                -30542.
No. Observations:               28238   AIC:                         6.112e+04
Df Residuals:                   28220   BIC:                         6.127e+04
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
======================================================================================================
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Intercept                              6.9276      0.089     77.586      0.000       6.753       7.103
education_level[T.Doctoral Degree]    -0.4486      0.040    -11.136      0.000      -0.528      -0.370
education_level[T.Graduate Degree]    -0.0574      0.009     -6.195      0.000      -0.076      -0.039
education_level[T.High School]        -0.1923      0.082     -2.347      0.019      -0.353      -0.032
gender[T.Male]                        -0.2537      0.010    -24.638      0.000      -0.274      -0.234
data_type[T.Survey]                    0.0351      0.019      1.891      0.059      -0.001       0.071
shipley_vocab                          0.0008      0.001      0.569      0.569      -0.002       0.003
shipley_abs                            0.0304      0.002     16.978      0.000       0.027       0.034
openness                               0.0649      0.008      8.316      0.000       0.050       0.080
conscientiousness                     -0.1828      0.008    -22.854      0.000      -0.198      -0.167
extraversion                          -0.0395      0.008     -5.229      0.000      -0.054      -0.025
agreeableness                          0.1816      0.008     21.706      0.000       0.165       0.198
neuroticism                            0.2325      0.009     24.489      0.000       0.214       0.251
pos_affect                             0.0026      0.001      2.347      0.019       0.000       0.005
neg_affect                            -0.0205      0.001    -14.661      0.000      -0.023      -0.018
stai_trait                             0.0002      0.001      0.263      0.792      -0.002       0.002
psqi                                  -0.0923      0.002    -46.906      0.000      -0.096      -0.088
age                                   -0.0045      0.001     -8.211      0.000      -0.006      -0.003
==============================================================================
Omnibus:                     6346.605   Durbin-Watson:                   0.372
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            42756.332
Skew:                          -0.913   Prob(JB):                         0.00
Kurtosis:                       8.745   Cond. No.                     1.63e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.63e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [227]:
# Establish baseline using LIFE EVENT VARIABLES.
# Only SURVEY data is used.
baseline_life_event_variables_sr(dependent_variable).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  sleep   R-squared:                       0.149
Model:                            OLS   Adj. R-squared:                  0.149
Method:                 Least Squares   F-statistic:                     387.4
Date:                Tue, 20 Sep 2022   Prob (F-statistic):               0.00
Time:                        22:38:10   Log-Likelihood:                -29329.
No. Observations:               26516   AIC:                         5.868e+04
Df Residuals:                   26503   BIC:                         5.879e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
=================================================================================================
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept                         7.5660      0.049    155.129      0.000       7.470       7.662
Anticipation[T.Unanticipated]    -0.1655      0.012    -13.723      0.000      -0.189      -0.142
LifeEventFamily[T.Health]        -0.9261      0.024    -39.166      0.000      -0.972      -0.880
LifeEventFamily[T.Local]         -0.6524      0.030    -22.068      0.000      -0.710      -0.594
LifeEventFamily[T.Personal]      -0.5628      0.019    -30.164      0.000      -0.599      -0.526
LifeEventFamily[T.School]        -0.5859      0.031    -18.925      0.000      -0.647      -0.525
LifeEventFamily[T.Work]          -0.6316      0.022    -28.207      0.000      -0.676      -0.588
status[T.Ongoing]                -0.4519      0.011    -39.551      0.000      -0.474      -0.430
valence                          -0.0350      0.006     -6.348      0.000      -0.046      -0.024
recency                          -0.0003   5.67e-05     -4.645      0.000      -0.000      -0.000
Intimacy                          0.0323      0.012      2.776      0.006       0.009       0.055
Scope                             0.1465      0.013     11.548      0.000       0.122       0.171
significance_label                0.0100      0.002      4.885      0.000       0.006       0.014
==============================================================================
Omnibus:                     3420.208   Durbin-Watson:                   0.283
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            15144.365
Skew:                          -0.570   Prob(JB):                         0.00
Kurtosis:                       6.523   Cond. No.                     7.57e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.57e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [228]:
# Establish baseline using LIFE EVENT VARIABLES.
# Only SOCIAL MEDIA data is used.
baseline_life_event_variables_sm(dependent_variable).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  sleep   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     1.421
Date:                Tue, 20 Sep 2022   Prob (F-statistic):              0.157
Time:                        22:38:10   Log-Likelihood:                -2317.0
No. Observations:                1722   AIC:                             4658.
Df Residuals:                    1710   BIC:                             4723.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
=================================================================================================
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept                         7.5389      0.388     19.425      0.000       6.778       8.300
Anticipation[T.Unanticipated]    -0.0366      0.063     -0.581      0.561      -0.160       0.087
LifeEventFamily[T.Health]         0.3132      0.266      1.179      0.239      -0.208       0.834
LifeEventFamily[T.Local]          0.1448      0.279      0.519      0.604      -0.403       0.692
LifeEventFamily[T.Personal]       0.2405      0.246      0.977      0.329      -0.242       0.724
LifeEventFamily[T.School]         0.2904      0.281      1.034      0.301      -0.260       0.841
LifeEventFamily[T.Work]           0.2066      0.267      0.775      0.439      -0.316       0.730
status[T.ongoing]                -0.0326      0.065     -0.502      0.616      -0.160       0.095
valence                          -0.0372      0.046     -0.816      0.415      -0.127       0.052
recency                          -0.0009      0.000     -3.182      0.001      -0.001      -0.000
Intimacy                          0.0265      0.057      0.463      0.643      -0.086       0.138
Scope                            -0.1079      0.079     -1.374      0.170      -0.262       0.046
==============================================================================
Omnibus:                      781.045   Durbin-Watson:                   0.917
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             7937.193
Skew:                          -1.858   Prob(JB):                         0.00
Kurtosis:                      12.840   Cond. No.                     2.07e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.07e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [229]:
# Establish baseline using both CONTROL and LIFE EVENT VARIABLES.
# Only SURVEY data is used.
regression_all_variables_sr(dependent_variable).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  sleep   R-squared:                       0.329
Model:                            OLS   Adj. R-squared:                  0.328
Method:                 Least Squares   F-statistic:                     463.6
Date:                Tue, 20 Sep 2022   Prob (F-statistic):               0.00
Time:                        22:38:11   Log-Likelihood:                -26184.
No. Observations:               26516   AIC:                         5.243e+04
Df Residuals:                   26487   BIC:                         5.266e+04
Df Model:                          28                                         
Covariance Type:            nonrobust                                         
======================================================================================================
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Intercept                              7.6720      0.094     81.703      0.000       7.488       7.856
education_level[T.Doctoral Degree]    -0.7456      0.044    -17.085      0.000      -0.831      -0.660
education_level[T.Graduate Degree]    -0.0819      0.009     -8.672      0.000      -0.100      -0.063
education_level[T.High School]         0.3188      0.326      0.979      0.328      -0.320       0.957
gender[T.Male]                        -0.2045      0.011    -18.902      0.000      -0.226      -0.183
Anticipation[T.Unanticipated]         -0.1816      0.011    -16.157      0.000      -0.204      -0.160
LifeEventFamily[T.Health]             -0.8964      0.022    -40.024      0.000      -0.940      -0.853
LifeEventFamily[T.Local]              -0.5894      0.027    -21.796      0.000      -0.642      -0.536
LifeEventFamily[T.Personal]           -0.4869      0.017    -28.491      0.000      -0.520      -0.453
LifeEventFamily[T.School]             -0.8333      0.030    -28.142      0.000      -0.891      -0.775
LifeEventFamily[T.Work]               -0.5246      0.020    -25.655      0.000      -0.565      -0.484
status[T.Ongoing]                     -0.3078      0.011    -27.463      0.000      -0.330      -0.286
shipley_vocab                          0.0021      0.001      1.613      0.107      -0.000       0.005
shipley_abs                            0.0142      0.002      7.597      0.000       0.011       0.018
openness                               0.1179      0.008     15.329      0.000       0.103       0.133
conscientiousness                     -0.2252      0.008    -27.689      0.000      -0.241      -0.209
extraversion                          -0.0452      0.008     -5.999      0.000      -0.060      -0.030
agreeableness                          0.1573      0.008     19.074      0.000       0.141       0.174
neuroticism                            0.2514      0.010     25.481      0.000       0.232       0.271
pos_affect                             0.0043      0.001      3.965      0.000       0.002       0.006
neg_affect                            -0.0313      0.001    -22.424      0.000      -0.034      -0.029
stai_trait                             0.0025      0.001      2.623      0.009       0.001       0.004
psqi                                  -0.0859      0.002    -43.329      0.000      -0.090      -0.082
age                                   -0.0004      0.001     -0.644      0.520      -0.001       0.001
valence                               -0.0546      0.005    -10.369      0.000      -0.065      -0.044
recency                               -0.0001    5.3e-05     -2.659      0.008      -0.000   -3.71e-05
Intimacy                              -0.1012      0.011     -9.313      0.000      -0.123      

In [230]:
# Establish baseline using both CONTROL and LIFE EVENT VARIABLES.
# Only SOCIAL MEDIA data is used.
regression_all_variables_sm(dependent_variable).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  sleep   R-squared:                       0.136
Model:                            OLS   Adj. R-squared:                  0.122
Method:                 Least Squares   F-statistic:                     9.882
Date:                Tue, 20 Sep 2022   Prob (F-statistic):           6.61e-38
Time:                        22:38:11   Log-Likelihood:                -2198.9
No. Observations:                1722   AIC:                             4454.
Df Residuals:                    1694   BIC:                             4606.
Df Model:                          27                                         
Covariance Type:            nonrobust                                         
======================================================================================================
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Intercept                              6.7952      0.530     12.814      0.000       5.755       7.835
education_level[T.Doctoral Degree]     0.2786      0.108      2.573      0.010       0.066       0.491
education_level[T.Graduate Degree]    -0.0334      0.052     -0.639      0.523      -0.136       0.069
education_level[T.High School]        -0.5801      0.115     -5.026      0.000      -0.806      -0.354
gender[T.Male]                         0.0825      0.050      1.634      0.102      -0.017       0.181
Anticipation[T.Unanticipated]         -0.0177      0.060     -0.297      0.766      -0.135       0.099
LifeEventFamily[T.Health]              0.5128      0.252      2.031      0.042       0.018       1.008
LifeEventFamily[T.Local]               0.2946      0.265      1.111      0.267      -0.225       0.814
LifeEventFamily[T.Personal]            0.3781      0.233      1.620      0.106      -0.080       0.836
LifeEventFamily[T.School]              0.3921      0.267      1.470      0.142      -0.131       0.915
LifeEventFamily[T.Work]                0.3479      0.253      1.375      0.169      -0.149       0.844
status[T.ongoing]                     -0.0318      0.062     -0.516      0.606      -0.153       0.089
shipley_vocab                          0.0542      0.008      7.089      0.000       0.039       0.069
shipley_abs                           -0.0498      0.009     -5.738      0.000      -0.067      -0.033
openness                               0.0011      0.046      0.025      0.980      -0.088       0.090
conscientiousness                      0.2866      0.044      6.567      0.000       0.201       0.372
extraversion                          -0.0710      0.042     -1.707      0.088      -0.153       0.011
agreeableness                          0.0523      0.045      1.150      0.250      -0.037       0.142
neuroticism                            0.0134      0.047      0.285      0.775      -0.079       0.105
pos_affect                            -0.0225      0.006     -3.993      0.000      -0.034      -0.011
neg_affect                             0.0061      0.007      0.859      0.391      -0.008       0.020
stai_trait                            -0.0032      0.005     -0.602      0.547      -0.014       0.007
psqi                                  -0.0726      0.010     -7.552      0.000      -0.091      -0.054
age                                   -0.0070      0.003     -2.683      0.007      -0.012      -0.002
valence                                0.0075      0.043      0.172      0.863      -0.078       0.093
recency                               -0.0009      0.000     -3.403      0.001      -0.001      -0.000
Intimacy                               0.0332      0.054      0.611      0.541      -0.073      

In [231]:
# Establish baseline using both CONTROL and LIFE EVENT VARIABLES.
# Data includes both survey and social media.
regression_all_variables(dependent_variable).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  sleep   R-squared:                       0.291
Model:                            OLS   Adj. R-squared:                  0.290
Method:                 Least Squares   F-statistic:                     398.9
Date:                Tue, 20 Sep 2022   Prob (F-statistic):               0.00
Time:                        22:38:12   Log-Likelihood:                -29003.
No. Observations:               28238   AIC:                         5.807e+04
Df Residuals:                   28208   BIC:                         5.831e+04
Df Model:                          29                                         
Covariance Type:            nonrobust                                         
======================================================================================================
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Intercept                              5.8075      0.071     82.076      0.000       5.669       5.946
education_level[T.Doctoral Degree]    -0.5819      0.038    -15.128      0.000      -0.657      -0.507
education_level[T.Graduate Degree]    -0.0809      0.009     -8.587      0.000      -0.099      -0.062
education_level[T.High School]        -0.1983      0.078     -2.551      0.011      -0.351      -0.046
gender[T.Male]                        -0.1991      0.011    -18.883      0.000      -0.220      -0.178
Anticipation[T.Unanticipated]         -0.1565      0.011    -14.007      0.000      -0.178      -0.135
LifeEventFamily[T.Health]             -0.8940      0.023    -39.421      0.000      -0.938      -0.850
LifeEventFamily[T.Local]              -0.5753      0.027    -21.114      0.000      -0.629      -0.522
LifeEventFamily[T.Personal]           -0.5015      0.018    -28.635      0.000      -0.536      -0.467
LifeEventFamily[T.School]             -0.7978      0.030    -26.803      0.000      -0.856      -0.739
LifeEventFamily[T.Work]               -0.5433      0.021    -26.220      0.000      -0.584      -0.503
status[T.Ongoing]                     -0.3254      0.011    -28.559      0.000      -0.348      -0.303
status[T.ended]                        1.8986      0.040     47.776      0.000       1.821       1.976
status[T.ongoing]                      1.8698      0.030     62.030      0.000       1.811       1.929
data_type[T.Survey]                    2.0391      0.026     77.290      0.000       1.987       2.091
shipley_vocab                          0.0050      0.001      3.818      0.000       0.002       0.008
shipley_abs                            0.0073      0.002      4.035      0.000       0.004       0.011
openness                               0.1065      0.008     14.054      0.000       0.092       0.121
conscientiousness                     -0.1784      0.008    -22.298      0.000      -0.194      -0.163
extraversion                          -0.0437      0.007     -5.850      0.000      -0.058      -0.029
agreeableness                          0.1400      0.008     17.149      0.000       0.124       0.156
neuroticism                            0.2183      0.009     23.083      0.000       0.200       0.237
pos_affect                             0.0008      0.001      0.711      0.477      -0.001       0.003
neg_affect                            -0.0260      0.001    -18.821      0.000      -0.029      -0.023
stai_trait                             0.0025      0.001      2.626      0.009       0.001       0.004
psqi                                  -0.0906      0.002    -46.818      0.000      -0.094      -0.087
age                                   -0.0012      0.001     -2.192      0.028      -0.002      

### ANXIETY

In [232]:
dependent_variable = 'anxiety'

In [233]:
# Establish baseline using CONTROL VARIABLES.
# Data includes both survey and social media.
baseline_control_variables(dependent_variable).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                anxiety   R-squared:                       0.300
Model:                            OLS   Adj. R-squared:                  0.299
Method:                 Least Squares   F-statistic:                     745.8
Date:                Tue, 20 Sep 2022   Prob (F-statistic):               0.00
Time:                        22:38:12   Log-Likelihood:                -18057.
No. Observations:               29636   AIC:                         3.615e+04
Df Residuals:                   29618   BIC:                         3.630e+04
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
======================================================================================================
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Intercept                              1.0453      0.054     19.264      0.000       0.939       1.152
education_level[T.Doctoral Degree]     0.2612      0.025     10.656      0.000       0.213       0.309
education_level[T.Graduate Degree]     0.1360      0.006     24.117      0.000       0.125       0.147
education_level[T.High School]        -0.4004      0.048     -8.264      0.000      -0.495      -0.305
gender[T.Male]                        -0.0022      0.006     -0.350      0.726      -0.014       0.010
data_type[T.Survey]                    0.0811      0.011      7.469      0.000       0.060       0.102
shipley_vocab                         -0.0093      0.001    -11.100      0.000      -0.011      -0.008
shipley_abs                           -0.0049      0.001     -4.519      0.000      -0.007      -0.003
openness                              -0.0002      0.005     -0.039      0.969      -0.010       0.009
conscientiousness                      0.0162      0.005      3.318      0.001       0.007       0.026
extraversion                           0.1192      0.005     25.875      0.000       0.110       0.128
agreeableness                          0.0368      0.005      7.174      0.000       0.027       0.047
neuroticism                            0.0981      0.006     17.008      0.000       0.087       0.109
pos_affect                            -0.0156      0.001    -23.270      0.000      -0.017      -0.014
neg_affect                             0.0156      0.001     18.377      0.000       0.014       0.017
stai_trait                             0.0088      0.001     15.382      0.000       0.008       0.010
psqi                                   0.0335      0.001     28.053      0.000       0.031       0.036
age                                   -0.0051      0.000    -15.488      0.000      -0.006      -0.004
==============================================================================
Omnibus:                     4545.636   Durbin-Watson:                   0.432
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            14670.642
Skew:                           0.787   Prob(JB):                         0.00
Kurtosis:                       6.067   Cond. No.                     1.62e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.62e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [234]:
# Establish baseline using LIFE EVENT VARIABLES.
# Only SURVEY data is used.
baseline_life_event_variables_sr(dependent_variable).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                anxiety   R-squared:                       0.015
Model:                            OLS   Adj. R-squared:                  0.015
Method:                 Least Squares   F-statistic:                     35.74
Date:                Tue, 20 Sep 2022   Prob (F-statistic):           1.34e-83
Time:                        22:38:13   Log-Likelihood:                -21370.
No. Observations:               27664   AIC:                         4.277e+04
Df Residuals:                   27651   BIC:                         4.287e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
=================================================================================================
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept                         1.6350      0.034     48.794      0.000       1.569       1.701
Anticipation[T.Unanticipated]     0.0424      0.008      5.041      0.000       0.026       0.059
LifeEventFamily[T.Health]        -0.0286      0.017     -1.714      0.087      -0.061       0.004
LifeEventFamily[T.Local]         -0.0191      0.021     -0.921      0.357      -0.060       0.022
LifeEventFamily[T.Personal]       0.0017      0.013      0.129      0.898      -0.024       0.027
LifeEventFamily[T.School]         0.1150      0.022      5.284      0.000       0.072       0.158
LifeEventFamily[T.Work]          -0.0150      0.016     -0.956      0.339      -0.046       0.016
status[T.Ongoing]                -0.0088      0.008     -1.091      0.275      -0.025       0.007
valence                           0.0233      0.004      6.050      0.000       0.016       0.031
recency                           0.0004   3.83e-05     10.863      0.000       0.000       0.000
Intimacy                          0.0463      0.008      5.695      0.000       0.030       0.062
Scope                            -0.0530      0.009     -5.993      0.000      -0.070      -0.036
significance_label               -0.0127      0.001     -8.823      0.000      -0.016      -0.010
==============================================================================
Omnibus:                     2379.139   Durbin-Watson:                   0.261
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3800.004
Skew:                           0.652   Prob(JB):                         0.00
Kurtosis:                       4.264   Cond. No.                     7.58e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.58e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [235]:
# Establish baseline using LIFE EVENT VARIABLES.
# Only SOCIAL MEDIA data is used.
baseline_life_event_variables_sm(dependent_variable).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                anxiety   R-squared:                       0.020
Model:                            OLS   Adj. R-squared:                  0.014
Method:                 Least Squares   F-statistic:                     3.568
Date:                Tue, 20 Sep 2022   Prob (F-statistic):           5.41e-05
Time:                        22:38:13   Log-Likelihood:                -1662.7
No. Observations:                1972   AIC:                             3349.
Df Residuals:                    1960   BIC:                             3416.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
=================================================================================================
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept                         1.5515      0.211      7.337      0.000       1.137       1.966
Anticipation[T.Unanticipated]     0.0504      0.036      1.416      0.157      -0.019       0.120
LifeEventFamily[T.Health]         0.2955      0.136      2.173      0.030       0.029       0.562
LifeEventFamily[T.Local]          0.0924      0.145      0.635      0.525      -0.193       0.378
LifeEventFamily[T.Personal]       0.0662      0.125      0.530      0.596      -0.179       0.311
LifeEventFamily[T.School]         0.1017      0.146      0.696      0.486      -0.185       0.388
LifeEventFamily[T.Work]           0.2415      0.137      1.765      0.078      -0.027       0.510
status[T.ongoing]                -0.0068      0.037     -0.184      0.854      -0.079       0.065
valence                           0.0179      0.026      0.699      0.485      -0.032       0.068
recency                       -1.537e-05      0.000     -0.099      0.921      -0.000       0.000
Intimacy                          0.0094      0.032      0.294      0.769      -0.054       0.072
Scope                             0.0101      0.044      0.228      0.820      -0.077       0.097
==============================================================================
Omnibus:                      444.113   Durbin-Watson:                   0.995
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1167.766
Skew:                           1.190   Prob(JB):                    2.65e-254
Kurtosis:                       5.924   Cond. No.                     1.92e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.92e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [236]:
# Establish baseline using both CONTROL and LIFE EVENT VARIABLES.
# Only SURVEY data is used.
regression_all_variables_sr(dependent_variable).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                anxiety   R-squared:                       0.319
Model:                            OLS   Adj. R-squared:                  0.318
Method:                 Least Squares   F-statistic:                     461.8
Date:                Tue, 20 Sep 2022   Prob (F-statistic):               0.00
Time:                        22:38:13   Log-Likelihood:                -16273.
No. Observations:               27664   AIC:                         3.260e+04
Df Residuals:                   27635   BIC:                         3.284e+04
Df Model:                          28                                         
Covariance Type:            nonrobust                                         
======================================================================================================
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Intercept                              1.1576      0.062     18.809      0.000       1.037       1.278
education_level[T.Doctoral Degree]     0.3434      0.029     11.916      0.000       0.287       0.400
education_level[T.Graduate Degree]     0.1384      0.006     22.312      0.000       0.126       0.151
education_level[T.High School]        -0.7291      0.218     -3.337      0.001      -1.157      -0.301
gender[T.Male]                        -0.0059      0.007     -0.836      0.403      -0.020       0.008
Anticipation[T.Unanticipated]          0.0225      0.007      3.073      0.002       0.008       0.037
LifeEventFamily[T.Health]              0.0563      0.015      3.807      0.000       0.027       0.085
LifeEventFamily[T.Local]               0.0522      0.018      2.931      0.003       0.017       0.087
LifeEventFamily[T.Personal]           -0.0097      0.011     -0.859      0.390      -0.032       0.012
LifeEventFamily[T.School]              0.1436      0.020      7.360      0.000       0.105       0.182
LifeEventFamily[T.Work]                0.0847      0.013      6.306      0.000       0.058       0.111
status[T.Ongoing]                     -0.0415      0.007     -5.590      0.000      -0.056      -0.027
shipley_vocab                         -0.0108      0.001    -12.522      0.000      -0.013      -0.009
shipley_abs                           -0.0074      0.001     -5.992      0.000      -0.010      -0.005
openness                               0.0016      0.005      0.324      0.746      -0.008       0.012
conscientiousness                      0.0267      0.005      4.995      0.000       0.016       0.037
extraversion                           0.1189      0.005     24.015      0.000       0.109       0.129
agreeableness                          0.0299      0.005      5.478      0.000       0.019       0.041
neuroticism                            0.1080      0.006     16.645      0.000       0.095       0.121
pos_affect                            -0.0189      0.001    -26.330      0.000      -0.020      -0.018
neg_affect                             0.0172      0.001     18.783      0.000       0.015       0.019
stai_trait                             0.0060      0.001      9.660      0.000       0.005       0.007
psqi                                   0.0393      0.001     30.357      0.000       0.037       0.042
age                                   -0.0051      0.000    -13.697      0.000      -0.006      -0.004
valence                                0.0178      0.003      5.157      0.000       0.011       0.025
recency                                0.0003   3.36e-05      8.556      0.000       0.000       0.000
Intimacy                               0.0699      0.007      9.811      0.000       0.056      

In [237]:
# Establish baseline using both CONTROL and LIFE EVENT VARIABLES.
# Only SOCIAL MEDIA data is used.
regression_all_variables_sm(dependent_variable).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                anxiety   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.249
Method:                 Least Squares   F-statistic:                     25.25
Date:                Tue, 20 Sep 2022   Prob (F-statistic):          8.55e-107
Time:                        22:38:14   Log-Likelihood:                -1385.8
No. Observations:                1972   AIC:                             2828.
Df Residuals:                    1944   BIC:                             2984.
Df Model:                          27                                         
Covariance Type:            nonrobust                                         
======================================================================================================
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Intercept                              0.5127      0.278      1.846      0.065      -0.032       1.057
education_level[T.Doctoral Degree]     0.0552      0.057      0.967      0.334      -0.057       0.167
education_level[T.Graduate Degree]     0.1256      0.027      4.567      0.000       0.072       0.179
education_level[T.High School]        -0.3540      0.061     -5.767      0.000      -0.474      -0.234
gender[T.Male]                        -0.0936      0.027     -3.512      0.000      -0.146      -0.041
Anticipation[T.Unanticipated]          0.0657      0.031      2.089      0.037       0.004       0.127
LifeEventFamily[T.Health]              0.2934      0.120      2.452      0.014       0.059       0.528
LifeEventFamily[T.Local]               0.1945      0.128      1.514      0.130      -0.057       0.446
LifeEventFamily[T.Personal]            0.1717      0.110      1.561      0.119      -0.044       0.388
LifeEventFamily[T.School]              0.2174      0.129      1.688      0.092      -0.035       0.470
LifeEventFamily[T.Work]                0.2509      0.121      2.081      0.038       0.014       0.487
status[T.ongoing]                      0.0105      0.033      0.322      0.747      -0.053       0.074
shipley_vocab                         -0.0049      0.004     -1.222      0.222      -0.013       0.003
shipley_abs                            0.0025      0.005      0.546      0.585      -0.006       0.011
openness                               0.0482      0.024      1.992      0.046       0.001       0.096
conscientiousness                      0.0449      0.023      1.950      0.051      -0.000       0.090
extraversion                          -0.0597      0.022     -2.705      0.007      -0.103      -0.016
agreeableness                          0.0214      0.024      0.895      0.371      -0.025       0.068
neuroticism                            0.0323      0.025      1.300      0.194      -0.016       0.081
pos_affect                            -0.0002      0.003     -0.063      0.950      -0.006       0.006
neg_affect                             0.0158      0.004      4.242      0.000       0.008       0.023
stai_trait                             0.0142      0.003      4.949      0.000       0.009       0.020
psqi                                   0.0088      0.005      1.749      0.080      -0.001       0.019
age                                   -0.0058      0.001     -4.277      0.000      -0.009      -0.003
valence                                0.0177      0.023      0.777      0.437      -0.027       0.062
recency                             5.702e-05      0.000      0.409      0.682      -0.000       0.000
Intimacy                               0.0165      0.028      0.583      0.560      -0.039      

In [238]:
# Establish baseline using both CONTROL and LIFE EVENT VARIABLES.
# Data includes both survey and social media.
regression_all_variables(dependent_variable).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                anxiety   R-squared:                       0.311
Model:                            OLS   Adj. R-squared:                  0.310
Method:                 Least Squares   F-statistic:                     461.1
Date:                Tue, 20 Sep 2022   Prob (F-statistic):               0.00
Time:                        22:38:14   Log-Likelihood:                -17814.
No. Observations:               29636   AIC:                         3.569e+04
Df Residuals:                   29606   BIC:                         3.594e+04
Df Model:                          29                                         
Covariance Type:            nonrobust                                         
======================================================================================================
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Intercept                              0.8044      0.045     17.894      0.000       0.716       0.892
education_level[T.Doctoral Degree]     0.2758      0.024     11.259      0.000       0.228       0.324
education_level[T.Graduate Degree]     0.1415      0.006     23.578      0.000       0.130       0.153
education_level[T.High School]        -0.4291      0.048     -8.902      0.000      -0.524      -0.335
gender[T.Male]                        -0.0170      0.007     -2.528      0.011      -0.030      -0.004
Anticipation[T.Unanticipated]          0.0254      0.007      3.592      0.000       0.012       0.039
LifeEventFamily[T.Health]              0.0542      0.015      3.728      0.000       0.026       0.083
LifeEventFamily[T.Local]               0.0421      0.017      2.417      0.016       0.008       0.076
LifeEventFamily[T.Personal]           -0.0103      0.011     -0.918      0.358      -0.032       0.012
LifeEventFamily[T.School]              0.1301      0.019      6.841      0.000       0.093       0.167
LifeEventFamily[T.Work]                0.0780      0.013      5.893      0.000       0.052       0.104
status[T.Ongoing]                     -0.0362      0.007     -4.940      0.000      -0.051      -0.022
status[T.ended]                        0.2333      0.025      9.439      0.000       0.185       0.282
status[T.ongoing]                      0.2254      0.019     11.940      0.000       0.188       0.262
data_type[T.Survey]                    0.3457      0.017     20.705      0.000       0.313       0.378
shipley_vocab                         -0.0110      0.001    -13.085      0.000      -0.013      -0.009
shipley_abs                           -0.0065      0.001     -5.715      0.000      -0.009      -0.004
openness                               0.0052      0.005      1.066      0.286      -0.004       0.015
conscientiousness                      0.0277      0.005      5.444      0.000       0.018       0.038
extraversion                           0.1103      0.005     23.207      0.000       0.101       0.120
agreeableness                          0.0284      0.005      5.436      0.000       0.018       0.039
neuroticism                            0.1001      0.006     16.624      0.000       0.088       0.112
pos_affect                            -0.0178      0.001    -25.690      0.000      -0.019      -0.016
neg_affect                             0.0175      0.001     19.858      0.000       0.016       0.019
stai_trait                             0.0067      0.001     11.114      0.000       0.006       0.008
psqi                                   0.0367      0.001     29.860      0.000       0.034       0.039
age                                   -0.0052      0.000    -15.006      0.000      -0.006      